# Importações
Aqui encontram-se todas as importações utilizadas neste documento

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.pipeline import Pipeline
from scipy.stats import uniform
from datetime import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import RandomizedSearchCV
from openpyxl import Workbook, load_workbook
from openpyxl.styles import PatternFill, Border, Side



# Funções implementadas
Como estes excertos de código serão utilizadas para cada algoritmo, decidiu-se encapsular as funções para evitar a repetição de código
- <strong>save_classification_metrics_to_excel</strong>: Esta função guarda todas as métricas obtidas num ficheiro <strong>.xlsx</strong>, que posteriormente será guardado na Pasta Results do nosso projeto
- <strong>plot_confusion_matrix</strong>: Gere as matrizes de confusão em gráfico e guarda-as na pasta Results
- <strong>print_and_save_classification_results</strong>: Calcula e imprime as seguintes métricas:
    - Accuracy
    - Precision
    - Recall
    - F1-Score
    - Matriz de confusão
    - Relatório de classificação

    Posteriormente é chamada a função <strong>save_classification_metrics_to_excel</strong>

In [3]:
from openpyxl import Workbook, load_workbook
from openpyxl.styles import PatternFill, Border, Side
from datetime import datetime

def save_classification_metrics_to_excel(conf_matrix_test, conf_matrix_train, report_test, report_train, accuracy_test, accuracy_train, precision_test, precision_train, recall_test, recall_train, f1_test, f1_train, execution_time, best_params, filename):
    try:
        # Carrega o arquivo Excel existente
        wb = load_workbook(filename)
        ws = wb.active

        # Encontra a próxima linha vazia na planilha
        next_row = ws.max_row + 2

        # Adiciona os dados na planilha
        ws.cell(row=next_row, column=1, value="Matriz de Confusão (Teste)")
        for i, row in enumerate(conf_matrix_test, start=1):
            for j, value in enumerate(row, start=1):
                ws.cell(row=next_row + i, column=j, value=value)

        ws.cell(row=next_row + len(conf_matrix_test) + 2, column=1, value="Matriz de Confusão (Treino)")
        for i, row in enumerate(conf_matrix_train, start=1):
            for j, value in enumerate(row, start=1):
                ws.cell(row=next_row + len(conf_matrix_test) + 2 + i, column=j, value=value)

        ws.cell(row=next_row + 2 * len(conf_matrix_test) + 4, column=1, value="Relatório de Classificação (Teste)")
        for i, line in enumerate(report_test.split('\n'), start=1):
            ws.cell(row=next_row + 2 * len(conf_matrix_test) + 4 + i, column=1, value=line)

        ws.cell(row=next_row + 2 * len(conf_matrix_test) + 4 + len(report_test.split('\n')) + 2, column=1, value="Relatório de Classificação (Treino)")
        for i, line in enumerate(report_train.split('\n'), start=1):
            ws.cell(row=next_row + 2 * len(conf_matrix_test) + 4 + len(report_test.split('\n')) + 2 + i, column=1, value=line)
        next_row += 7

        ws.cell(row=next_row + 2 * len(conf_matrix_test) + 4 + len(report_test.split('\n')) + 4 + 2, column=1, value="Acurácia (Teste)")
        ws.cell(row=next_row + 2 * len(conf_matrix_test) + 4 + len(report_test.split('\n')) + 4 + 2, column=2, value=accuracy_test)

        ws.cell(row=next_row + 2 * len(conf_matrix_test) + 4 + len(report_test.split('\n')) + 6 + 2, column=1, value="Acurácia (Treino)")
        ws.cell(row=next_row + 2 * len(conf_matrix_test) + 4 + len(report_test.split('\n')) + 6 + 2, column=2, value=accuracy_train)

        ws.cell(row=next_row + 2 * len(conf_matrix_test) + 4 + len(report_test.split('\n')) + 8 + 2, column=1, value="Precision (Teste)")
        ws.cell(row=next_row + 2 * len(conf_matrix_test) + 4 + len(report_test.split('\n')) + 8 + 2, column=2, value=precision_test)

        ws.cell(row=next_row + 2 * len(conf_matrix_test) + 4 + len(report_test.split('\n')) + 10 + 2, column=1, value="Precision (Treino)")
        ws.cell(row=next_row + 2 * len(conf_matrix_test) + 4 + len(report_test.split('\n')) + 10 + 2, column=2, value=precision_train)

        ws.cell(row=next_row + 2 * len(conf_matrix_test) + 4 + len(report_test.split('\n')) + 12 + 2, column=1, value="Recall (Teste)")
        ws.cell(row=next_row + 2 * len(conf_matrix_test) + 4 + len(report_test.split('\n')) + 12 + 2, column=2, value=recall_test)

        ws.cell(row=next_row + 2 * len(conf_matrix_test) + 4 + len(report_test.split('\n')) + 14 + 2, column=1, value="Recall (Treino)")
        ws.cell(row=next_row + 2 * len(conf_matrix_test) + 4 + len(report_test.split('\n')) + 14 + 2, column=2, value=recall_train)

        ws.cell(row=next_row + 2 * len(conf_matrix_test) + 4 + len(report_test.split('\n')) + 16 + 2, column=1, value="F1-score (Teste)")
        ws.cell(row=next_row + 2 * len(conf_matrix_test) + 4 + len(report_test.split('\n')) + 16 + 2, column=2, value=f1_test)

        ws.cell(row=next_row + 2 * len(conf_matrix_test) + 4 + len(report_test.split('\n')) + 18 + 2, column=1, value="F1-score (Treino)")
        ws.cell(row=next_row + 2 * len(conf_matrix_test) + 4 + len(report_test.split('\n')) + 18 + 2, column=2, value=f1_train)

        ws.cell(row=next_row + 2 * len(conf_matrix_test) + 4 + len(report_test.split('\n')) + 20 + 2, column=1, value="Tempo de execução")
        ws.cell(row=next_row + 2 * len(conf_matrix_test) + 4 + len(report_test.split('\n')) + 20 + 2, column=2, value=str(execution_time))

        ws.cell(row=next_row + 2 * len(conf_matrix_test) + 4 + len(report_test.split('\n')) + 22 + 2, column=1, value="Melhores Parâmetros")
        ws.cell(row=next_row + 2 * len(conf_matrix_test) + 4 + len(report_test.split('\n')) + 22 + 2, column=2, value=str(best_params))

        # Salva as alterações no arquivo
        wb.save(filename)
        print("Dados adicionados com sucesso!")

    except FileNotFoundError:
        # Se o arquivo não existir, cria uma nova planilha e adiciona os dados
        print(f"O arquivo '{filename}' não foi encontrado. Criando um novo arquivo...")
        wb = Workbook()
        ws = wb.active

        # Encontra a próxima linha vazia na planilha
        next_row = ws.max_row + 2

        # Adiciona os dados na planilha
        ws.cell(row=next_row, column=1, value="Matriz de Confusão (Teste)")
        for i, row in enumerate(conf_matrix_test, start=1):
            for j, value in enumerate(row, start=1):
                ws.cell(row=next_row + i, column=j, value=value)

        ws.cell(row=next_row + len(conf_matrix_test) + 2, column=1, value="Matriz de Confusão (Treino)")
        for i, row in enumerate(conf_matrix_train, start=1):
            for j, value in enumerate(row, start=1):
                ws.cell(row=next_row + len(conf_matrix_test) + 2 + i, column=j, value=value)

        ws.cell(row=next_row + 2 * len(conf_matrix_test) + 4, column=1, value="Relatório de Classificação (Teste)")
        for i, line in enumerate(report_test.split('\n'), start=1):
            ws.cell(row=next_row + 2 * len(conf_matrix_test) + 4 + i, column=1, value=line)

        ws.cell(row=next_row + 2 * len(conf_matrix_test) + 4 + len(report_test.split('\n')) + 2, column=1, value="Relatório de Classificação (Treino)")
        for i, line in enumerate(report_train.split('\n'), start=1):
            ws.cell(row=next_row + 2 * len(conf_matrix_test) + 4 + len(report_test.split('\n')) + 2 + i, column=1, value=line)
        next_row += 7

        ws.cell(row=next_row + 2 * len(conf_matrix_test) + 4 + len(report_test.split('\n')) + 4 + 2, column=1, value="Acurácia (Teste)")
        ws.cell(row=next_row + 2 * len(conf_matrix_test) + 4 + len(report_test.split('\n')) + 4 + 2, column=2, value=accuracy_test)

        ws.cell(row=next_row + 2 * len(conf_matrix_test) + 4 + len(report_test.split('\n')) + 6 + 2, column=1, value="Acurácia (Treino)")
        ws.cell(row=next_row + 2 * len(conf_matrix_test) + 4 + len(report_test.split('\n')) + 6 + 2, column=2, value=accuracy_train)

        ws.cell(row=next_row + 2 * len(conf_matrix_test) + 4 + len(report_test.split('\n')) + 8 + 2, column=1, value="Precision (Teste)")
        ws.cell(row=next_row + 2 * len(conf_matrix_test) + 4 + len(report_test.split('\n')) + 8 + 2, column=2, value=precision_test)

        ws.cell(row=next_row + 2 * len(conf_matrix_test) + 4 + len(report_test.split('\n')) + 10 + 2, column=1, value="Precision (Treino)")
        ws.cell(row=next_row + 2 * len(conf_matrix_test) + 4 + len(report_test.split('\n')) + 10 + 2, column=2, value=precision_train)

        ws.cell(row=next_row + 2 * len(conf_matrix_test) + 4 + len(report_test.split('\n')) + 12 + 2, column=1, value="Recall (Teste)")
        ws.cell(row=next_row + 2 * len(conf_matrix_test) + 4 + len(report_test.split('\n')) + 12 + 2, column=2, value=recall_test)

        ws.cell(row=next_row + 2 * len(conf_matrix_test) + 4 + len(report_test.split('\n')) + 14 + 2, column=1, value="Recall (Treino)")
        ws.cell(row=next_row + 2 * len(conf_matrix_test) + 4 + len(report_test.split('\n')) + 14 + 2, column=2, value=recall_train)

        ws.cell(row=next_row + 2 * len(conf_matrix_test) + 4 + len(report_test.split('\n')) + 16 + 2, column=1, value="F1-score (Teste)")
        ws.cell(row=next_row + 2 * len(conf_matrix_test) + 4 + len(report_test.split('\n')) + 16 + 2, column=2, value=f1_test)

        ws.cell(row=next_row + 2 * len(conf_matrix_test) + 4 + len(report_test.split('\n')) + 18 + 2, column=1, value="F1-score (Treino)")
        ws.cell(row=next_row + 2 * len(conf_matrix_test) + 4 + len(report_test.split('\n')) + 18 + 2, column=2, value=f1_train)

        ws.cell(row=next_row + 2 * len(conf_matrix_test) + 4 + len(report_test.split('\n')) + 20 + 2, column=1, value="Tempo de execução")
        ws.cell(row=next_row + 2 * len(conf_matrix_test) + 4 + len(report_test.split('\n')) + 20 + 2, column=2, value=str(execution_time))

        ws.cell(row=next_row + 2 * len(conf_matrix_test) + 4 + len(report_test.split('\n')) + 22 + 2, column=1, value="Melhores Parâmetros")
        ws.cell(row=next_row + 2 * len(conf_matrix_test) + 4 + len(report_test.split('\n')) + 22 + 2, column=2, value=str(best_params))

        # Salva as alterações no arquivo
        wb.save(filename)
        print("Dados adicionados com sucesso!")




# Função para plotar a matriz de confusão
def plot_confusion_matrix(conf_matrix, title, filename):
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False)
    plt.title(title)
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.savefig(filename)
    plt.close()

def print_and_save_classification_results(model,y_test, y_pred, y_train, y_pred_train, execution_time,best_params, filename):
    # Calcular a matriz de confusão para o conjunto de teste e de treino
    conf_matrix_test = confusion_matrix(y_test, y_pred)
    conf_matrix_train = confusion_matrix(y_train, y_pred_train)

    # Calcular as métricas para o conjunto de teste
    report_test = classification_report(y_test, y_pred)
    accuracy_test = accuracy_score(y_test, y_pred)
    precision_test = precision_score(y_test, y_pred)
    recall_test = recall_score(y_test, y_pred)
    f1_test = f1_score(y_test, y_pred)

    # Calcular as métricas para o conjunto de treino
    report_train = classification_report(y_train, y_pred_train)
    accuracy_train = accuracy_score(y_train, y_pred_train)
    precision_train = precision_score(y_train, y_pred_train)
    recall_train = recall_score(y_train, y_pred_train)
    f1_train = f1_score(y_train, y_pred_train)

    # Imprimir os resultados
    print("Matriz de Confusão (Teste):")
    print(conf_matrix_test)
    print("\nMatriz de Confusão (Treino):")
    print(conf_matrix_train)

    print("\nRelatório de Classificação (Teste):")
    print(report_test)
    print("\nAcurácia (Teste):", accuracy_test)
    print("Precision (Teste):", precision_test)
    print("Recall (Teste):", recall_test)
    print("F1-score (Teste):", f1_test)

    print("\nRelatório de Classificação (Treino):")
    print(report_train)
    print("\nAcurácia (Treino):", accuracy_train)
    print("Precision (Treino):", precision_train)
    print("Recall (Treino):", recall_train)
    print("F1-score (Treino):", f1_train)

    print("\nMelhores Parâmetros:", best_params)

    print("\nTempo de execução:", execution_time)

    # Plotar e salvar as matrizes de confusão
    plot_confusion_matrix(conf_matrix_test, f'Matriz de Confusão (Teste) - {model}', f'Results/confusion_matrix_test_{model}.png')
    plot_confusion_matrix(conf_matrix_train, f'Matriz de Confusão (Treino) -{model}', f'Results/confusion_matrix_train_{model}.png')

    # Salvar os resultados em um arquivo Excel
    save_classification_metrics_to_excel(conf_matrix_test, conf_matrix_train, report_test, report_train, accuracy_test, accuracy_train, precision_test, precision_train, recall_test, recall_train, f1_test, f1_train, execution_time,best_params, filename)

# Ler Ficheiro e divisão dos dados
Começamos por realizar a leitura dos do dataset e dividir os dados em treino e teste (80-20)

In [4]:
data_url = 'Datasets/total_treated.csv'

# Read the dataset using pandas
data = pd.read_csv(data_url, delimiter=';', header=0, encoding='utf-8')

# Dividir os dados em features e target
X = data.drop(columns=['label'])
y = data['label']

# Dividir os dados em conjunto de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Logistic Regression
A regressão logística é um método estatístico utilizado para modelar a relação entre uma variável dependente binária (ou categórica) e uma ou mais variáveis independentes (ou preditoras). Em vez de prever valores contínuos, como a regressão linear, a regressão logística prevê a probabilidade de uma determinada classe ou evento ocorrer.

In [5]:
# Definir os hiperparâmetros para otimizar
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear', 'saga']
}

# Criar o modelo de regressão logística
log_reg = LogisticRegression(max_iter=1000)

# Criar o objeto RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=log_reg, param_distributions=param_grid, n_iter=10, cv=5, verbose=2, random_state=42, n_jobs=-1)

# Treinar o modelo usando RandomizedSearchCV
start_time = datetime.now()
random_search.fit(X_train, y_train)
end_time = datetime.now()
execution_time = end_time - start_time

# Obter os melhores parâmetros
best_params = random_search.best_params_

# Obter as previsões para o conjunto de teste
y_pred = random_search.predict(X_test)

# Obter as previsões para o conjunto de treino
y_pred_train = random_search.predict(X_train)

# Imprimir métrica e Salvar os resultados em um arquivo .xlsx
print_and_save_classification_results("Regressão Logistica",y_test, y_pred, y_train, y_pred_train, execution_time,best_params,filename="Results/Logistic_Regression.xlsx")

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Matriz de Confusão (Teste):
[[3994    2]
 [   0 4783]]

Matriz de Confusão (Treino):
[[16021     1]
 [    1 19093]]

Relatório de Classificação (Teste):
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      3996
         1.0       1.00      1.00      1.00      4783

    accuracy                           1.00      8779
   macro avg       1.00      1.00      1.00      8779
weighted avg       1.00      1.00      1.00      8779


Acurácia (Teste): 0.9997721836200023
Precision (Teste): 0.9995820271682341
Recall (Teste): 1.0
F1-score (Teste): 0.9997909698996655

Relatório de Classificação (Treino):
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     16022
         1.0       1.00      1.00      1.00     19094

    accuracy                           1.00     35116
   macro avg       1.00      1.00      1.00     35116
weighted avg    

# Random Forest Model
O Random Forest é um algoritmo de aprendizagem automática de conjunto (ensemble learning) utilizado tanto para classificação como para regressão. O Random Forest constrói uma "floresta" de muitas árvores de decisão e combina as suas previsões para obter um resultado mais preciso e robusto.

In [6]:
# Definir os hiperparâmetros para otimizar
param_grid = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Criar o modelo Random Forest
rf = RandomForestClassifier()

# Criar o objeto RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_grid, n_iter=10, cv=5, verbose=2, random_state=42, n_jobs=-1)

# Treinar o modelo usando RandomizedSearchCV
start_time = datetime.now()
random_search.fit(X_train, y_train)
end_time = datetime.now()
execution_time = end_time - start_time

# Obter os melhores parâmetros
best_params = random_search.best_params_

# Obter as previsões para o conjunto de teste
y_pred = random_search.predict(X_test)

# Obter as previsões para o conjunto de treino
y_pred_train = random_search.predict(X_train)

# Imprimir métrica e Salvar os resultados em um arquivo .xlsx
print_and_save_classification_results("Random Forest",y_test, y_pred, y_train, y_pred_train, execution_time,best_params,filename="Results/Random_Forest.xlsx")

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Matriz de Confusão (Teste):
[[3996    0]
 [   0 4783]]

Matriz de Confusão (Treino):
[[16022     0]
 [    0 19094]]

Relatório de Classificação (Teste):
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      3996
         1.0       1.00      1.00      1.00      4783

    accuracy                           1.00      8779
   macro avg       1.00      1.00      1.00      8779
weighted avg       1.00      1.00      1.00      8779


Acurácia (Teste): 1.0
Precision (Teste): 1.0
Recall (Teste): 1.0
F1-score (Teste): 1.0

Relatório de Classificação (Treino):
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     16022
         1.0       1.00      1.00      1.00     19094

    accuracy                           1.00     35116
   macro avg       1.00      1.00      1.00     35116
weighted avg       1.00      1.00      1.00     35116


Acurá

# Neural Network
Uma Neural Network é um modelo de aprendizagem automática inspirado no funcionamento do cérebro humano. É composta por camadas de unidades de processamento, chamadas neurónios, que estão interligadas. As redes neuronais são utilizadas para reconhecer padrões complexos e aprender a partir de grandes quantidades de dados.

In [7]:
# Definir os hiperparâmetros para otimizar
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (150,)],
    'activation': ['relu', 'tanh', 'logistic'],
    'solver': ['adam', 'sgd'],
    'alpha': [0.0001, 0.001, 0.01]
    }

# Criar o modelo de Rede Neural
neural_net = MLPClassifier(max_iter=1000)

# Criar o objeto RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=neural_net, param_distributions=param_grid, n_iter=10, cv=5, verbose=2, random_state=42, n_jobs=-1)

# Treinar o modelo usando RandomizedSearchCV
start_time = datetime.now()
random_search.fit(X_train, y_train)
end_time = datetime.now()
execution_time = end_time - start_time

# Obter os melhores parâmetros
best_params = random_search.best_params_

# Obter as previsões para o conjunto de teste
y_pred = random_search.predict(X_test)

# Obter as previsões para o conjunto de treino
y_pred_train = random_search.predict(X_train)

# Imprimir métrica e Salvar os resultados em um arquivo .xlsx
print_and_save_classification_results("Neural Network",y_test, y_pred, y_train, y_pred_train, execution_time,best_params,filename="Results/Neural_Network.xlsx")

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Matriz de Confusão (Teste):
[[3914   82]
 [   4 4779]]

Matriz de Confusão (Treino):
[[15721   301]
 [   16 19078]]

Relatório de Classificação (Teste):
              precision    recall  f1-score   support

         0.0       1.00      0.98      0.99      3996
         1.0       0.98      1.00      0.99      4783

    accuracy                           0.99      8779
   macro avg       0.99      0.99      0.99      8779
weighted avg       0.99      0.99      0.99      8779


Acurácia (Teste): 0.9902038956600979
Precision (Teste): 0.9831310429952684
Recall (Teste): 0.99916370478779
F1-score (Teste): 0.9910825383658233

Relatório de Classificação (Treino):
              precision    recall  f1-score   support

         0.0       1.00      0.98      0.99     16022
         1.0       0.98      1.00      0.99     19094

    accuracy                           0.99     35116
   macro avg       0.99      0.99      0.99     35116
wei